In [1]:
# Loading Libraries
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('GPU Available!')
else: 
    print('No GPU available.')

GPU Available!


In [3]:
# Loading Dataset
f = open('dataset.txt', 'r', encoding='utf-8')
sonnets_dataset = f.read().split('<eos>')
del sonnets_dataset[-1]
dataset_string = ''
for line in sonnets_dataset:
    dataset_string = dataset_string + line
# Encoding Dataset 
chars = tuple(set(dataset_string))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in dataset_string])
# encoded[:100]


In [4]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [5]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [6]:
# define and print the net
n_hidden=512
n_layers=3

mm = CharRNN(chars, n_hidden, n_layers)
print(mm)

CharRNN(
  (lstm): LSTM(88, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=88, bias=True)
)


In [7]:
# #Loading the whole Model
mm = torch.load('model_try.pt')

In [8]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [9]:
def sample(net, size, prime='whose conduct others', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [10]:
print(sample(mm, 1000, prime='said she', top_k=5))

said she would have strip  she seeming , and thy souls who should have loved at his shore  that there as starry time , and thou at least the seas  till hate and where that to a mean  so marching and to shun him wouldst i strives thee , and some threst of sight  and to her souls to that more sorrow , though and from the belle  as streamlet while they startling well to might  in a thousand laws , and so desired  and worthier ,— when they show , as any silent than my life  surrives to the walfs a the sand or true  with their tongue the wind , i simplish 'd are 
that it might blame , and would tell thou do but  we how both to that mould the flow  and the life , since with fortune to the winds  to love in some souls with mass , they lifted to be lay 
such looks , and trump of soul to than a stern  alone though the blood stream the frame  with any long things seem 'd , with all her lesson shall should spare  such walls on ships they let the sorrowed sea  this words are stern and frozen world

In [11]:
print(sample(mm, 1000, prime='who is', top_k=5))

who is  when the service , and when the light and least  the sea , of the seas , and and whose truth , then forth the whall  there thou to shine and will they lie  that more with traitorian lie  when the boundless figure should beheld  in their song , and to should make  with heaven , this that sorrow a stead  and fallen , when to life , without a calm and for the sea  the blaze when time as thou art so  and looks to these declare  to mak 'd i steede them when i shall man 's from the black  and love , that i then seem 'd  when his song through the strong  a monarch of the ball  the word and legions , touching sorry was  and the soul of thy lives with souls are like  and stirs it is to her the lovely blest 
this shame thou shalt no more , in me to took  and why in sins and then i should to meet thee all her bare 
this light and solitude the winds  this wondering forms ,— i was it that the sun  the sheep of hearts and ways  that we and tremble , and thou art  i said that so , and follows

In [16]:
print(sample(mm, 1000, prime='when they stumbled', top_k=5))

when they stumbled  of her sings were with the brides of sound  of still who had to late , and said , and we alive to make a ship  of many a memory , and when i seal  and so that the shadow , and with sheerly strain  alone thy steps all the blind shadowy field  and then when the strengths are from the strife  still the strangers and wings , and this was thoughts , and then  what i she would have stood the ships  of soundlight of the sun whose honour of might hear  whose worthy want , the sun that speaks to feel  think in my feeble spirit , and the sea  the wood , or laughing , was a stone  so trav'l of the soul  the while things of the sea  to see thy sighs , befrom thy soul  all silver wondrous song , who loved and shore 
a fountain song , thou wast , and to the beauty 's fire  with the soul of all a silent light  the little tasking wander of the flame  who wonders and still , the water stars  wilt that what thou wouldst thought a many a soul 
we crowd thyself the face , which throws 